In [9]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Загрузка данных
df = pd.read_csv('2023-11-07_Opros_pervokursnikov_3.csv')  

# Преобразование категориальных признаков в числовой формат
label_encoder = LabelEncoder()
df['Ответ_вопроса_1'] = label_encoder.fit_transform(df['Какое из утверждений наиболее близко описывает ваше отношение к участию в научных конференциях?'])
df['Ответ_вопроса_2'] = label_encoder.fit_transform(df['В какой тематике вы бы хотели участвовать в научной конференции?'])

# Кластеризация с использованием Agglomerative Clustering
n_clusters = 4  # Задаем количество кластеров
agg_cluster = AgglomerativeClustering(n_clusters=n_clusters)
labels_agg = agg_cluster.fit_predict(df[['Ответ_вопроса_1']])

# Добавление столбца 'Cluster' с метками кластеров в исходный датасет
df['Cluster'] = labels_agg

# Вывод результатов
df.to_csv(f'test_dead_df.csv', index=False)


In [10]:
df['Target_Cluster'] = 0

# Условия для включения ответов в целевой кластер
target_conditions = (
    ((df['Какое из утверждений наиболее близко описывает ваше отношение к участию в научных конференциях?'] == 'Мне интересно, но я ещё не участвовал') & 
     df['В какой тематике вы бы хотели участвовать в научной конференции?'].isin(['Любая тема интересует', 'Технологии и инновации'])),
    ((df['Какое из утверждений наиболее близко описывает ваше отношение к участию в научных конференциях?'] == 'Я активно ищу возможности для участия') & 
     df['В какой тематике вы бы хотели участвовать в научной конференции?'].isin(['Любая тема интересует', 'Технологии и инновации']))
    # достаточно спорный/ситуативный выбор - поэтому закомментирован
    # ((df['Какое из утверждений наиболее близко описывает ваше отношение к участию в научных конференциях?'] == 'Я не знаю, что такое научные конференции.') & 
    #  df['В какой тематике вы бы хотели участвовать в научной конференции?'].isin(['Любая тема интересует', 'Технологии и инновации']))
)

# Применяем условия к каждому кластеру
df.loc[target_conditions[0], 'Target_Cluster'] = 1
df.loc[target_conditions[1], 'Target_Cluster'] = 1
#df.loc[target_conditions[2], 'Target_Cluster'] = 1


In [11]:
test = df[['Какое из утверждений наиболее близко описывает ваше отношение к участию в научных конференциях?',
                    'В какой тематике вы бы хотели участвовать в научной конференции?', 'Cluster', 'Target_Cluster']]
test.to_csv('test_dead1.csv', index=False)

cluster_counts = test['Cluster'].value_counts()
custom_cluster_counts = test['Target_Cluster'].value_counts()

print("Standard Clusters:")
print(cluster_counts)

print("\nTarget Cluster:")
print(custom_cluster_counts)

Standard Clusters:
Cluster
0    80
2    51
1    11
3     8
Name: count, dtype: int64

Target Cluster:
Target_Cluster
0    94
1    56
Name: count, dtype: int64


In [12]:
cluster_distribution = test.groupby(['Какое из утверждений наиболее близко описывает ваше отношение к участию в научных конференциях?',
                                            'В какой тематике вы бы хотели участвовать в научной конференции?',
                                            'Cluster', 'Target_Cluster']).size().reset_index(name='Counts')

total_responses = cluster_distribution['Counts'].sum()
cluster_distribution['Percentage'] = (cluster_distribution['Counts'] / total_responses) * 100

cluster_distribution.to_csv('test_dead2.csv', index=False)